In [1]:
"""
    Debugging Notes:
    Penalty Values:
    
    Undercurtail: 10000 (min on time)   [Already okay, never violated]
    Overcurtail: 400000 (max off time)   [Less than 80k 7 instances max, at 80k 4 instances, at 100k 3 instances, 2 instances at 150k, 1 instance at 350k, no violations at 400k]
    Reqcurtail: 10000 (min curtail per TS) [Not yet re-tested]
    Excesscurtail: 1000 per kW [Not yet re-tested]
    Continuity: 10000 [Not yet re-tested]
    
"""



# Import modules
import numpy as np
import timeit

# Import sphere function as objective function
from pyswarms.utils.functions.single_obj import sphere as f

# Import backend modules
import pyswarms.backend as P
from pyswarms.backend.topology import Star
#import pyswarms as ps
#import pyswarms.discrete.binary as PSO

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
my_topology = Star() # The Topology Class
my_options = {'c1': 2, 'c2': 2, 'w': 2}
c1 = 2
c2 = 2
w = 2
#set dimension to 20 loads x 16 timeslots = 320
my_swarm = P.create_swarm(n_particles=30, dimensions=320, binary=True, discrete=True, options=my_options) # The Swarm Class

#Initialize cost to array of inf for max value, to_append is manually set below. Create temp storade for pbest cost as well
x = 0
temp_pbest_cost = []
to_append = np.array([float('inf')])

for x in range(my_swarm.n_particles):
    my_swarm.pbest_cost = np.append(my_swarm.pbest_cost, to_append)
    my_swarm.current_cost = np.append(my_swarm.current_cost, to_append)
    temp_pbest_cost = np.append(temp_pbest_cost, to_append)




#Printing Values for debugging
#print('The following are the attributes of our swarm: {}'.format(my_swarm.__dict__.keys()))
#print("# of particles:",my_swarm.n_particles)
#print("dimensions:",my_swarm.dimensions)
#print("pos \n",my_swarm.position)
#print("velo \n",my_swarm.velocity)
#print("current cost \n",my_swarm.current_cost)
#print("pbest pos \n",my_swarm.pbest_pos)
#print("pbest cost \n",my_swarm.pbest_cost)
#print("best cost \n",my_swarm.best_cost)

In [3]:
#Position computation, BPSO computation for position

def _compute_position(swarm):
    for i in range(swarm.n_particles):                  
        for j in range(my_swarm.dimensions):
            if swarm.velocity[i][j] < np.random.uniform(low=0.0, high=1.0):
                swarm.position[i][j] = 0
            else:
                swarm.position[i][j] = 1
    return swarm.position

In [4]:
#Velocity computation, BPSO computation for velocity

def _compute_velocity(swarm,c1,c2,w):
    a = np.multiply(swarm.velocity,w)
    b = np.multiply(np.subtract(swarm.pbest_pos,swarm.position),c1*np.random.uniform(low=0.0, high=1.0))
    c = np.multiply(np.subtract(swarm.best_pos,swarm.position),c2*np.random.uniform(low=0.0, high=1.0))
    swarm.velocity = a + b + c

    return 1 / (1 + np.exp(-swarm.velocity))

In [5]:
#Taken from pyswarms.backend.operators
def _compute_pbest(swarm,temp_pbest_cost):
    try:
        # Infer dimensions from positions
        dimensions = swarm.dimensions
        
        # Create a 1-D and 2-D mask based from comparisons
        # Creating 1-D mask
        mask_cost = swarm.current_cost < temp_pbest_cost
        #print("maskcost:\n",mask_cost)
        
        # Extension of 1-D mask to 2-D mask
        mask_pos = np.repeat(mask_cost[:, np.newaxis], dimensions, axis=1)
        #print("maskpos:\n",mask_pos)
        
        # Apply masks, copy into 2nd variable if 3rd variable fits conditions
        new_pbest_pos = np.where(~mask_pos, swarm.pbest_pos, swarm.position)
        new_pbest_cost = np.where(~mask_cost, temp_pbest_cost, swarm.current_cost)
        
        #print("swarm current pos:",swarm.position)
        #print("new swarm pbest pos:",new_pbest_pos)
        #print("new swarm pbest cost:",new_pbest_cost)
        
    except AttributeError:
        rep.logger.exception(
            "Please pass a Swarm class. You passed {}".format(type(swarm))
        )
        raise
    else:
        return (new_pbest_pos, new_pbest_cost)

In [6]:
#penalty for curtailing more than necessary (lighter penalty)
def overcurtail(x):
    
    #timeslotrange = 320 elements per particle divided by 20 loads
    timeslotrange = 16
    
    penaltyvalue = 400000
    
    """penaltyvalue = """
    
    
    totalpenalty = 0
    
    #divide array into 20 loads x 16 timeslots
    L0 = x[0:timeslotrange]
    L1 = x[timeslotrange:(timeslotrange*2)]
    L2 = x[(timeslotrange*2):(timeslotrange*3)]
    L3 = x[(timeslotrange*3):(timeslotrange*4)]
    L4 = x[(timeslotrange*4):(timeslotrange*5)]
    L5 = x[(timeslotrange*5):(timeslotrange*6)]
    L6 = x[(timeslotrange*6):(timeslotrange*7)]
    L7 = x[(timeslotrange*7):(timeslotrange*8)]
    L8 = x[(timeslotrange*8):(timeslotrange*9)]
    L9 = x[(timeslotrange*9):(timeslotrange*10)]
    L10 = x[(timeslotrange*10):(timeslotrange*11)]
    L11 = x[(timeslotrange*11):(timeslotrange*12)]
    L12 = x[(timeslotrange*12):(timeslotrange*13)]
    L13 = x[(timeslotrange*13):(timeslotrange*14)]
    L14 = x[(timeslotrange*14):(timeslotrange*15)]
    L15 = x[(timeslotrange*15):(timeslotrange*16)]
    L16 = x[(timeslotrange*16):(timeslotrange*17)]
    L17 = x[(timeslotrange*17):(timeslotrange*18)]
    L18 = x[(timeslotrange*18):(timeslotrange*19)]
    L19 = x[(timeslotrange*19):(timeslotrange*20)]
    
    #invert array 1 and 0, count 1s to find number of 0s
    inv = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
    L0 = np.subtract(inv,L0)
    L1 = np.subtract(inv,L1)
    L2 = np.subtract(inv,L2)
    L3 = np.subtract(inv,L3)
    L4 = np.subtract(inv,L4)
    L5 = np.subtract(inv,L5)
    L6 = np.subtract(inv,L6)
    L7 = np.subtract(inv,L7)
    L8 = np.subtract(inv,L8)
    L9 = np.subtract(inv,L9)
    L10 = np.subtract(inv,L10)
    L11 = np.subtract(inv,L11)
    L12 = np.subtract(inv,L12)
    L13 = np.subtract(inv,L13)
    L14 = np.subtract(inv,L14)
    L15 = np.subtract(inv,L15)
    L16 = np.subtract(inv,L16)
    L17 = np.subtract(inv,L17)
    L18 = np.subtract(inv,L18)
    L19 = np.subtract(inv,L19)
    
    #set maximum off time per load here
    limitL0 = 4
    limitL1 = 4
    limitL2 = 4
    limitL3 = 4
    limitL4 = 4
    limitL5 = 4
    limitL6 = 3
    limitL7 = 3
    limitL8 = 3
    limitL9 = 4
    limitL10 = 3
    limitL11 = 3
    limitL12 = 4
    limitL13 = 3
    limitL14 = 3
    limitL15 = 3
    limitL16 = 4
    limitL17 = 4
    limitL18 = 4
    limitL19 = 4
    
    #count for off time here, add 1 penalty value if violated
    if np.sum(L0) > limitL0:
        totalpenalty += 1
    if np.sum(L1) > limitL1:
        totalpenalty += 1
    if np.sum(L2) > limitL2:
        totalpenalty += 1
    if np.sum(L3) > limitL3:
        totalpenalty += 1
    if np.sum(L4) > limitL4:
        totalpenalty += 1
    if np.sum(L5) > limitL5:
        totalpenalty += 1
    if np.sum(L6) > limitL6:
        totalpenalty += 1
    if np.sum(L7) > limitL7:
        totalpenalty += 1
    if np.sum(L8) > limitL8:
        totalpenalty += 1
    if np.sum(L9) > limitL9:
        totalpenalty += 1
    if np.sum(L10) > limitL10:
        totalpenalty += 1
    if np.sum(L11) > limitL11:
        totalpenalty += 1
    if np.sum(L12) > limitL12:
        totalpenalty += 1
    if np.sum(L13) > limitL13:
        totalpenalty += 1
    if np.sum(L14) > limitL14:
        totalpenalty += 1
    if np.sum(L15) > limitL15:
        totalpenalty += 1
    if np.sum(L16) > limitL16:
        totalpenalty += 1
    if np.sum(L17) > limitL17:
        totalpenalty += 1
    if np.sum(L18) > limitL18:
        totalpenalty += 1
    if np.sum(L19) > limitL19:
        totalpenalty += 1
        
    """
    excessoff0 = np.sum(L0) - limitL0
    if excessoff0 < 0:
        excessoff0 = 0
    excessoff1 = np.sum(L1) - limitL1
    if excessoff1 < 0:
        excessoff1 = 0
    excessoff2 = np.sum(L2) - limitL2
    if excessoff2 < 0:
        excessoff2 = 0
    excessoff3 = np.sum(L3) - limitL3
    if excessoff3 < 0:
        excessoff3 = 0
    excessoff4 = np.sum(L4) - limitL4
    if excessoff4 < 0:
        excessoff4 = 0
    excessoff5 = np.sum(L5) - limitL5
    if excessoff5 < 0:
        excessoff5 = 0
    excessoff6 = np.sum(L6) - limitL6
    if excessoff6 < 0:
        excessoff6 = 0
    excessoff7 = np.sum(L7) - limitL7
    if excessoff7 < 0:
        excessoff7 = 0
    excessoff8 = np.sum(L8) - limitL8
    if excessoff8 < 0:
        excessoff8 = 0
    excessoff9 = np.sum(L9) - limitL9
    if excessoff9 < 0:
        excessoff9 = 0
    excessoff10 = np.sum(L10) - limitL10
    if excessoff10 < 0:
        excessoff10 = 0
    excessoff11 = np.sum(L11) - limitL11
    if excessoff11 < 0:
        excessoff11 = 0
    excessoff12 = np.sum(L12) - limitL12
    if excessoff12 < 0:
        excessoff12 = 0
    excessoff13 = np.sum(L13) - limitL13
    if excessoff13 < 0:
        excessoff13 = 0
    excessoff14 = np.sum(L14) - limitL14
    if excessoff14 < 0:
        excessoff14 = 0
    excessoff15 = np.sum(L15) - limitL15
    if excessoff15 < 0:
        excessoff15 = 0
    excessoff16 = np.sum(L16) - limitL16
    if excessoff16 < 0:
        excessoff16 = 0
    excessoff17 = np.sum(L17) - limitL17
    if excessoff17 < 0:
        excessoff17 = 0
    excessoff18 = np.sum(L18) - limitL18
    if excessoff18 < 0:
        excessoff18 = 0
    excessoff19 = np.sum(L19) - limitL19
    if excessoff19 < 0:
        excessoff19 = 0
    
    totalpenalty = excessoff0 + excessoff1 + excessoff2 + excessoff3 + excessoff4 + excessoff5 + excessoff6 + excessoff7 + excessoff8 + excessoff9 +\
    excessoff10 + excessoff11 + excessoff12 + excessoff13 + excessoff14 + excessoff15 + excessoff16 + excessoff17 + excessoff18 + excessoff19
    """
    
    totalpenalty = totalpenalty*penaltyvalue
    
    return totalpenalty

In [7]:
#penalty for undercurtailing (should be high multiplier to ensure output is not all zero)
def undercurtail(x):
    
    #timeslotrange = 320 elements per particle divided by 20 loads
    timeslotrange = 16
    penaltyvalue = 10000
    totalpenalty = 0
    
    #divide array into 20 loads x 16 timeslots
    L0 = x[0:timeslotrange]
    L1 = x[timeslotrange:(timeslotrange*2)]
    L2 = x[(timeslotrange*2):(timeslotrange*3)]
    L3 = x[(timeslotrange*3):(timeslotrange*4)]
    L4 = x[(timeslotrange*4):(timeslotrange*5)]
    L5 = x[(timeslotrange*5):(timeslotrange*6)]
    L6 = x[(timeslotrange*6):(timeslotrange*7)]
    L7 = x[(timeslotrange*7):(timeslotrange*8)]
    L8 = x[(timeslotrange*8):(timeslotrange*9)]
    L9 = x[(timeslotrange*9):(timeslotrange*10)]
    L10 = x[(timeslotrange*10):(timeslotrange*11)]
    L11 = x[(timeslotrange*11):(timeslotrange*12)]
    L12 = x[(timeslotrange*12):(timeslotrange*13)]
    L13 = x[(timeslotrange*13):(timeslotrange*14)]
    L14 = x[(timeslotrange*14):(timeslotrange*15)]
    L15 = x[(timeslotrange*15):(timeslotrange*16)]
    L16 = x[(timeslotrange*16):(timeslotrange*17)]
    L17 = x[(timeslotrange*17):(timeslotrange*18)]
    L18 = x[(timeslotrange*18):(timeslotrange*19)]
    L19 = x[(timeslotrange*19):(timeslotrange*20)]
    
    #set minimum on time per load here
    limitL0 = 2
    limitL1 = 2
    limitL2 = 2
    limitL3 = 2
    limitL4 = 2
    limitL5 = 2
    limitL6 = 1
    limitL7 = 1
    limitL8 = 1
    limitL9 = 1
    limitL10 = 1
    limitL11 = 3
    limitL12 = 3
    limitL13 = 3
    limitL14 = 3
    limitL15 = 3
    limitL16 = 2
    limitL17 = 2
    limitL18 = 2
    limitL19 = 2
    
    #count for ontime here, add 1 penalty value if violated
    if np.sum(L0) < limitL0:
        totalpenalty += 1
    if np.sum(L1) < limitL1:
        totalpenalty += 1
    if np.sum(L2) < limitL2:
        totalpenalty += 1
    if np.sum(L3) < limitL3:
        totalpenalty += 1
    if np.sum(L4) < limitL4:
        totalpenalty += 1
    if np.sum(L5) < limitL5:
        totalpenalty += 1
    if np.sum(L6) < limitL6:
        totalpenalty += 1
    if np.sum(L7) < limitL7:
        totalpenalty += 1
    if np.sum(L8) < limitL8:
        totalpenalty += 1
    if np.sum(L9) < limitL9:
        totalpenalty += 1
    if np.sum(L10) < limitL10:
        totalpenalty += 1
    if np.sum(L11) < limitL11:
        totalpenalty += 1
    if np.sum(L12) < limitL12:
        totalpenalty += 1
    if np.sum(L13) < limitL13:
        totalpenalty += 1
    if np.sum(L14) < limitL14:
        totalpenalty += 1
    if np.sum(L15) < limitL15:
        totalpenalty += 1
    if np.sum(L16) < limitL16:
        totalpenalty += 1
    if np.sum(L17) < limitL17:
        totalpenalty += 1
    if np.sum(L18) < limitL18:
        totalpenalty += 1
    if np.sum(L19) < limitL19:
        totalpenalty += 1
    
    totalpenalty = totalpenalty*penaltyvalue
    
    return totalpenalty

In [8]:
#penalty for not meeting required curtailment per timeslot
def reqcurtail(x):
    #penaltyvalue = 1000000
    penaltyvalue = 10000
    totalpenalty = 0
    
    #set requirement for each TS in kW here
    TS0limit = 110
    TS1limit = 220
    TS2limit = 455
    TS3limit = 680
    TS4limit = 770
    TS5limit = 800
    TS6limit = 750
    TS7limit = 640
    TS8limit = 590
    TS9limit = 610
    TS10limit = 660
    TS11limit = 680
    TS12limit = 570
    TS13limit = 410
    TS14limit = 230
    TS15limit = 135
    
    #set consumption for each load in kW here
    cons0 = 320
    cons1 = 200
    cons2 = 80
    cons3 = 84
    cons4 = 100
    cons5 = 160
    cons6 = 100
    cons7 = 60
    cons8 = 200
    cons9 = 40
    cons10 = 40
    cons11 = 72
    cons12 = 140
    cons13 = 80
    cons14 = 40
    cons15 = 180
    cons16 = 180
    cons17 = 160
    cons18 = 60
    cons19 = 50
    
    #set self generated energy for each load in kW here
    self0 = 0
    self1 = 0
    self2 = 0
    self3 = 0
    self4 = 0
    self5 = 0
    self6 = 0
    self7 = 0
    self8 = 0
    self9 = 0
    self10 = 0
    self11 = 0
    self12 = 0
    self13 = 0
    self14 = 0
    self15 = 0
    self16 = 0
    self17 = 0
    self18 = 0
    self19 = 0
    
    consumption = [cons0-self0, cons1-self1, cons2-self2, cons3-self3, cons4-self4, cons5-self5, cons6-self6, cons7-self7, cons8-self8, cons9-self9, cons10-self10, cons11-self11, cons12-self12, cons13-self13, cons14-self14, cons15-self15, cons16-self16, cons17-self17, cons18-self18, cons19-self19]
    
    #cut particles into timeslots here, from TS0 to TS15. Note: 320 = 20 loads x 16 timeslots
    TS0 = x[0:320:16]
    TS1 = x[1:320:16]
    TS2 = x[2:320:16]
    TS3 = x[3:320:16]
    TS4 = x[4:320:16]
    TS5 = x[5:320:16]
    TS6 = x[6:320:16]
    TS7 = x[7:320:16]
    TS8 = x[8:320:16]
    TS9 = x[9:320:16]
    TS10 = x[10:320:16]
    TS11 = x[11:320:16]
    TS12 = x[12:320:16]
    TS13 = x[13:320:16]
    TS14 = x[14:320:16]
    TS15 = x[15:320:16]
    
    #invert array 1 and 0, count 1s to find number of 0s
    #inv = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
    inv = np.ones(20, dtype=int)
    
    TS0 = np.subtract(inv,TS0)
    TS1 = np.subtract(inv,TS1)
    TS2 = np.subtract(inv,TS2)
    TS3 = np.subtract(inv,TS3)
    TS4 = np.subtract(inv,TS4)
    TS5 = np.subtract(inv,TS5)
    TS6 = np.subtract(inv,TS6)
    TS7 = np.subtract(inv,TS7)
    TS8 = np.subtract(inv,TS8)
    TS9 = np.subtract(inv,TS9)
    TS10 = np.subtract(inv,TS10)
    TS11 = np.subtract(inv,TS11)
    TS12 = np.subtract(inv,TS12)
    TS13 = np.subtract(inv,TS13)
    TS14 = np.subtract(inv,TS14)
    TS15 = np.subtract(inv,TS15)
    
    #check for instances of not meeting curtailment requirement
    if np.sum(np.multiply(TS0,consumption)) < TS0limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS1,consumption)) < TS1limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS2,consumption)) < TS2limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS3,consumption)) < TS3limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS4,consumption)) < TS4limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS5,consumption)) < TS5limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS6,consumption)) < TS6limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS7,consumption)) < TS7limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS8,consumption)) < TS8limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS9,consumption)) < TS9limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS10,consumption)) < TS10limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS11,consumption)) < TS11limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS12,consumption)) < TS12limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS13,consumption)) < TS13limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS14,consumption)) < TS14limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS15,consumption)) < TS15limit:
        totalpenalty += 1
    
    totalpenalty = totalpenalty*penaltyvalue
    
    return totalpenalty

In [9]:
def excesscurtail(x):
    
    #penalty value is for every kW in excess of required curtailment
    penaltyvalue = 1000
    totalpenalty = 0
    
    #set requirement for each TS in kW here
    TS0limit = 110
    TS1limit = 220
    TS2limit = 455
    TS3limit = 680
    TS4limit = 770
    TS5limit = 800
    TS6limit = 750
    TS7limit = 640
    TS8limit = 590
    TS9limit = 610
    TS10limit = 660
    TS11limit = 680
    TS12limit = 570
    TS13limit = 410
    TS14limit = 230
    TS15limit = 135
    
    #set consumption for each load in kW here
    cons0 = 320
    cons1 = 200
    cons2 = 80
    cons3 = 84
    cons4 = 100
    cons5 = 160
    cons6 = 100
    cons7 = 60
    cons8 = 200
    cons9 = 40
    cons10 = 40
    cons11 = 72
    cons12 = 140
    cons13 = 80
    cons14 = 40
    cons15 = 180
    cons16 = 180
    cons17 = 160
    cons18 = 60
    cons19 = 50
    
    #set self generated energy for each load in kW here
    self0 = 0
    self1 = 0
    self2 = 0
    self3 = 0
    self4 = 0
    self5 = 0
    self6 = 0
    self7 = 0
    self8 = 0
    self9 = 0
    self10 = 0
    self11 = 0
    self12 = 0
    self13 = 0
    self14 = 0
    self15 = 0
    self16 = 0
    self17 = 0
    self18 = 0
    self19 = 0
    
    consumption = [cons0-self0, cons1-self1, cons2-self2, cons3-self3, cons4-self4, cons5-self5, cons6-self6, cons7-self7, cons8-self8, cons9-self9, cons10-self10, cons11-self11, cons12-self12, cons13-self13, cons14-self14, cons15-self15, cons16-self16, cons17-self17, cons18-self18, cons19-self19]
    
    #cut particles into timeslots here, from TS0 to TS15. Note: 320 = 20 loads x 16 timeslots
    TS0 = x[0:320:16]
    TS1 = x[1:320:16]
    TS2 = x[2:320:16]
    TS3 = x[3:320:16]
    TS4 = x[4:320:16]
    TS5 = x[5:320:16]
    TS6 = x[6:320:16]
    TS7 = x[7:320:16]
    TS8 = x[8:320:16]
    TS9 = x[9:320:16]
    TS10 = x[10:320:16]
    TS11 = x[11:320:16]
    TS12 = x[12:320:16]
    TS13 = x[13:320:16]
    TS14 = x[14:320:16]
    TS15 = x[15:320:16]
    
    #invert to find off times
    inv = np.ones(20, dtype=int)
    TS0 = np.subtract(inv,TS0)
    TS1 = np.subtract(inv,TS1)
    TS2 = np.subtract(inv,TS2)
    TS3 = np.subtract(inv,TS3)
    TS4 = np.subtract(inv,TS4)
    TS5 = np.subtract(inv,TS5)
    TS6 = np.subtract(inv,TS6)
    TS7 = np.subtract(inv,TS7)
    TS8 = np.subtract(inv,TS8)
    TS9 = np.subtract(inv,TS9)
    TS10 = np.subtract(inv,TS10)
    TS11 = np.subtract(inv,TS11)
    TS12 = np.subtract(inv,TS12)
    TS13 = np.subtract(inv,TS13)
    TS14 = np.subtract(inv,TS14)
    TS15 = np.subtract(inv,TS15)
    
    
    excess0 = np.sum(np.multiply(TS0,consumption)) - TS0limit
    excess1 = np.sum(np.multiply(TS1,consumption)) - TS1limit
    excess2 = np.sum(np.multiply(TS2,consumption)) - TS2limit
    excess3 = np.sum(np.multiply(TS3,consumption)) - TS3limit
    excess4 = np.sum(np.multiply(TS4,consumption)) - TS4limit
    excess5 = np.sum(np.multiply(TS5,consumption)) - TS5limit
    excess6 = np.sum(np.multiply(TS6,consumption)) - TS6limit
    excess7 = np.sum(np.multiply(TS7,consumption)) - TS7limit
    excess8 = np.sum(np.multiply(TS8,consumption)) - TS8limit
    excess9 = np.sum(np.multiply(TS9,consumption)) - TS9limit
    excess10 = np.sum(np.multiply(TS10,consumption)) - TS10limit
    excess11 = np.sum(np.multiply(TS11,consumption)) - TS11limit
    excess12 = np.sum(np.multiply(TS12,consumption)) - TS12limit
    excess13 = np.sum(np.multiply(TS13,consumption)) - TS13limit
    excess14 = np.sum(np.multiply(TS14,consumption)) - TS14limit
    excess15 = np.sum(np.multiply(TS15,consumption)) - TS15limit
    """
    if excess0 < 0:
        excess0 = 0
    if excess1 < 0:
        excess1 = 0
    if excess2 < 0:
        excess2 = 0
    if excess3 < 0:
        excess3 = 0
    if excess4 < 0:
        excess4 = 0
    if excess5 < 0:
        excess5 = 0
    if excess6 < 0:
        excess6 = 0
    if excess7 < 0:
        excess7 = 0
    if excess8 < 0:
        excess8 = 0
    if excess9 < 0:
        excess9 = 0
    if excess10 < 0:
        excess10 = 0
    if excess11 < 0:
        excess11 = 0
    if excess12 < 0:
        excess12 = 0
    if excess13 < 0:
        excess13 = 0
    if excess14 < 0:
        excess14 = 0
    if excess15 < 0:
        excess15 = 0
    """
    
    excess0 = abs(excess0)
    excess1 = abs(excess1)
    excess2 = abs(excess2)
    excess3 = abs(excess3)
    excess4 = abs(excess4)
    excess5 = abs(excess5)
    excess6 = abs(excess6)
    excess7 = abs(excess7)
    excess8 = abs(excess8)
    excess9 = abs(excess9)
    excess10 = abs(excess10)
    excess11 = abs(excess11)
    excess12 = abs(excess12)
    excess13 = abs(excess13)
    excess14 = abs(excess14)
    excess15 = abs(excess15)
    
    totalexcess = excess0 + excess1 + excess2 + excess3 + excess4 + excess5 + excess6 + excess7 + excess8 + excess9 +\
    excess10 + excess11 + excess12 + excess13 + excess14 + excess15
    
    totalpenalty = totalexcess*penaltyvalue
    
    return totalpenalty

In [10]:
#function to make sure the curtailments are in continuous chunks, ideally one long curtailment
def continuity(x):
    #timeslotrange = 320 elements per particle divided by 20 loads
    timeslotrange = 16
    #penaltyvalue = 1000000
    penaltyvalue = 10000
    totalpenalty = 0
    
    #divide array into 20 loads x 16 timeslots
    L0 = x[0:timeslotrange]
    L1 = x[timeslotrange:(timeslotrange*2)]
    L2 = x[(timeslotrange*2):(timeslotrange*3)]
    L3 = x[(timeslotrange*3):(timeslotrange*4)]
    L4 = x[(timeslotrange*4):(timeslotrange*5)]
    L5 = x[(timeslotrange*5):(timeslotrange*6)]
    L6 = x[(timeslotrange*6):(timeslotrange*7)]
    L7 = x[(timeslotrange*7):(timeslotrange*8)]
    L8 = x[(timeslotrange*8):(timeslotrange*9)]
    L9 = x[(timeslotrange*9):(timeslotrange*10)]
    L10 = x[(timeslotrange*10):(timeslotrange*11)]
    L11 = x[(timeslotrange*11):(timeslotrange*12)]
    L12 = x[(timeslotrange*12):(timeslotrange*13)]
    L13 = x[(timeslotrange*13):(timeslotrange*14)]
    L14 = x[(timeslotrange*14):(timeslotrange*15)]
    L15 = x[(timeslotrange*15):(timeslotrange*16)]
    L16 = x[(timeslotrange*16):(timeslotrange*17)]
    L17 = x[(timeslotrange*17):(timeslotrange*18)]
    L18 = x[(timeslotrange*18):(timeslotrange*19)]
    L19 = x[(timeslotrange*19):(timeslotrange*20)]
    
    
    #if there are more than 1 transitions from 1 to 0, not one long interrupt, add penalty 
    
    if np.count_nonzero(np.diff(L0)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L1)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L2)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L3)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L4)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L5)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L6)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L7)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L8)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L9)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L10)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L11)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L12)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L13)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L14)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L15)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L16)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L17)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L18)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L19)==-1) > 1:
        totalpenalty += 1
    
    totalpenalty = totalpenalty*penaltyvalue
    
    return totalpenalty

In [11]:
#test function, 2D version, to be implemented per element (Element is 1D array)
def obj_func(x):
    
    #timeslotrange = 320 elements per particle divided by 20 loads
    timeslotrange = 16
    
    #set costs per kW here
    DUcost = 10
    SGEcost = 5
    
    #set consumption for each load in kW here
    cons0 = 320
    cons1 = 200
    cons2 = 80
    cons3 = 84
    cons4 = 100
    cons5 = 160
    cons6 = 100
    cons7 = 60
    cons8 = 200
    cons9 = 40
    cons10 = 40
    cons11 = 72
    cons12 = 140
    cons13 = 80
    cons14 = 40
    cons15 = 180
    cons16 = 180
    cons17 = 160
    cons18 = 60
    cons19 = 50
    
    #set self generated energy for each load in kW here
    self0 = 0
    self1 = 0
    self2 = 0
    self3 = 0
    self4 = 0
    self5 = 0
    self6 = 0
    self7 = 0
    self8 = 0
    self9 = 0
    self10 = 0
    self11 = 0
    self12 = 0
    self13 = 0
    self14 = 0
    self15 = 0
    self16 = 0
    self17 = 0
    self18 = 0
    self19 = 0

    #divide array into 20 loads x 16 timeslots
    L0 = x[0:timeslotrange]
    L1 = x[timeslotrange:(timeslotrange*2)]
    L2 = x[(timeslotrange*2):(timeslotrange*3)]
    L3 = x[(timeslotrange*3):(timeslotrange*4)]
    L4 = x[(timeslotrange*4):(timeslotrange*5)]
    L5 = x[(timeslotrange*5):(timeslotrange*6)]
    L6 = x[(timeslotrange*6):(timeslotrange*7)]
    L7 = x[(timeslotrange*7):(timeslotrange*8)]
    L8 = x[(timeslotrange*8):(timeslotrange*9)]
    L9 = x[(timeslotrange*9):(timeslotrange*10)]
    L10 = x[(timeslotrange*10):(timeslotrange*11)]
    L11 = x[(timeslotrange*11):(timeslotrange*12)]
    L12 = x[(timeslotrange*12):(timeslotrange*13)]
    L13 = x[(timeslotrange*13):(timeslotrange*14)]
    L14 = x[(timeslotrange*14):(timeslotrange*15)]
    L15 = x[(timeslotrange*15):(timeslotrange*16)]
    L16 = x[(timeslotrange*16):(timeslotrange*17)]
    L17 = x[(timeslotrange*17):(timeslotrange*18)]
    L18 = x[(timeslotrange*18):(timeslotrange*19)]
    L19 = x[(timeslotrange*19):(timeslotrange*20)]
    
    f =  np.sum(np.array(L0)*DUcost*(cons0-self0)) + np.sum(np.array(L0)*SGEcost*(self0)) +\
    np.sum(np.array(L1)*DUcost*(cons1-self1)) + np.sum(np.array(L1)*SGEcost*(self1)) +\
    np.sum(np.array(L2)*DUcost*(cons2-self2)) + np.sum(np.array(L2)*SGEcost*(self2)) +\
    np.sum(np.array(L3)*DUcost*(cons3-self3)) + np.sum(np.array(L3)*SGEcost*(self3)) +\
    np.sum(np.array(L4)*DUcost*(cons4-self4)) + np.sum(np.array(L4)*SGEcost*(self4)) +\
    np.sum(np.array(L5)*DUcost*(cons5-self5)) + np.sum(np.array(L5)*SGEcost*(self5)) +\
    np.sum(np.array(L6)*DUcost*(cons6-self6)) + np.sum(np.array(L6)*SGEcost*(self6)) +\
    np.sum(np.array(L7)*DUcost*(cons7-self7)) + np.sum(np.array(L7)*SGEcost*(self7)) +\
    np.sum(np.array(L8)*DUcost*(cons8-self8)) + np.sum(np.array(L8)*SGEcost*(self8)) +\
    np.sum(np.array(L9)*DUcost*(cons9-self9)) + np.sum(np.array(L9)*SGEcost*(self9)) +\
    np.sum(np.array(L10)*DUcost*(cons10-self10)) + np.sum(np.array(L10)*SGEcost*(self10)) +\
    np.sum(np.array(L11)*DUcost*(cons11-self11)) + np.sum(np.array(L11)*SGEcost*(self11)) +\
    np.sum(np.array(L12)*DUcost*(cons12-self12)) + np.sum(np.array(L12)*SGEcost*(self12)) +\
    np.sum(np.array(L13)*DUcost*(cons13-self13)) + np.sum(np.array(L13)*SGEcost*(self13)) +\
    np.sum(np.array(L14)*DUcost*(cons14-self14)) + np.sum(np.array(L14)*SGEcost*(self14)) +\
    np.sum(np.array(L15)*DUcost*(cons15-self15)) + np.sum(np.array(L15)*SGEcost*(self15)) +\
    np.sum(np.array(L16)*DUcost*(cons16-self16)) + np.sum(np.array(L16)*SGEcost*(self16)) +\
    np.sum(np.array(L17)*DUcost*(cons17-self17)) + np.sum(np.array(L17)*SGEcost*(self17)) +\
    np.sum(np.array(L18)*DUcost*(cons18-self18)) + np.sum(np.array(L18)*SGEcost*(self18)) +\
    np.sum(np.array(L19)*DUcost*(cons19-self19)) + np.sum(np.array(L19)*SGEcost*(self19)) +\
    overcurtail(x) + undercurtail(x) + reqcurtail(x) + excesscurtail(x) + continuity(x)
    
    return f


In [12]:
def checkvalidity(x):
     #set requirement for each TS in kW here
    TS0limit = 110
    TS1limit = 220
    TS2limit = 455
    TS3limit = 680
    TS4limit = 770
    TS5limit = 800
    TS6limit = 750
    TS7limit = 640
    TS8limit = 590
    TS9limit = 610
    TS10limit = 660
    TS11limit = 680
    TS12limit = 570
    TS13limit = 410
    TS14limit = 230
    TS15limit = 135
    
    #set consumption for each load in kW here
    cons0 = 320
    cons1 = 200
    cons2 = 80
    cons3 = 84
    cons4 = 100
    cons5 = 160
    cons6 = 100
    cons7 = 60
    cons8 = 200
    cons9 = 40
    cons10 = 40
    cons11 = 72
    cons12 = 140
    cons13 = 80
    cons14 = 40
    cons15 = 180
    cons16 = 180
    cons17 = 160
    cons18 = 60
    cons19 = 50
    
    #set self generated energy for each load in kW here
    self0 = 0
    self1 = 0
    self2 = 0
    self3 = 0
    self4 = 0
    self5 = 0
    self6 = 0
    self7 = 0
    self8 = 0
    self9 = 0
    self10 = 0
    self11 = 0
    self12 = 0
    self13 = 0
    self14 = 0
    self15 = 0
    self16 = 0
    self17 = 0
    self18 = 0
    self19 = 0
    
    consumption = [cons0-self0, cons1-self1, cons2-self2, cons3-self3, cons4-self4, cons5-self5, cons6-self6, cons7-self7, cons8-self8, cons9-self9, cons10-self10, cons11-self11, cons12-self12, cons13-self13, cons14-self14, cons15-self15, cons16-self16, cons17-self17, cons18-self18, cons19-self19]
    
    #parse to timeslots
    TS0 = x[0:320:16]
    TS1 = x[1:320:16]
    TS2 = x[2:320:16]
    TS3 = x[3:320:16]
    TS4 = x[4:320:16]
    TS5 = x[5:320:16]
    TS6 = x[6:320:16]
    TS7 = x[7:320:16]
    TS8 = x[8:320:16]
    TS9 = x[9:320:16]
    TS10 = x[10:320:16]
    TS11 = x[11:320:16]
    TS12 = x[12:320:16]
    TS13 = x[13:320:16]
    TS14 = x[14:320:16]
    TS15 = x[15:320:16]

    inv = np.ones(20, dtype=int)
    TS0 = np.subtract(inv,TS0)
    TS1 = np.subtract(inv,TS1)
    TS2 = np.subtract(inv,TS2)
    TS3 = np.subtract(inv,TS3)
    TS4 = np.subtract(inv,TS4)
    TS5 = np.subtract(inv,TS5)
    TS6 = np.subtract(inv,TS6)
    TS7 = np.subtract(inv,TS7)
    TS8 = np.subtract(inv,TS8)
    TS9 = np.subtract(inv,TS9)
    TS10 = np.subtract(inv,TS10)
    TS11 = np.subtract(inv,TS11)
    TS12 = np.subtract(inv,TS12)
    TS13 = np.subtract(inv,TS13)
    TS14 = np.subtract(inv,TS14)
    TS15 = np.subtract(inv,TS15)
    
    excess0 = np.sum(np.multiply(TS0,consumption)) - TS0limit
    excess1 = np.sum(np.multiply(TS1,consumption)) - TS1limit
    excess2 = np.sum(np.multiply(TS2,consumption)) - TS2limit
    excess3 = np.sum(np.multiply(TS3,consumption)) - TS3limit
    excess4 = np.sum(np.multiply(TS4,consumption)) - TS4limit
    excess5 = np.sum(np.multiply(TS5,consumption)) - TS5limit
    excess6 = np.sum(np.multiply(TS6,consumption)) - TS6limit
    excess7 = np.sum(np.multiply(TS7,consumption)) - TS7limit
    excess8 = np.sum(np.multiply(TS8,consumption)) - TS8limit
    excess9 = np.sum(np.multiply(TS9,consumption)) - TS9limit
    excess10 = np.sum(np.multiply(TS10,consumption)) - TS10limit
    excess11 = np.sum(np.multiply(TS11,consumption)) - TS11limit
    excess12 = np.sum(np.multiply(TS12,consumption)) - TS12limit
    excess13 = np.sum(np.multiply(TS13,consumption)) - TS13limit
    excess14 = np.sum(np.multiply(TS14,consumption)) - TS14limit
    excess15 = np.sum(np.multiply(TS15,consumption)) - TS15limit
    
    excessarray = [excess0, excess1, excess2, excess3, excess4, excess5, excess6, excess7, excess8, excess9, excess10, excess11, excess12, excess13, excess14, excess15]
    print("Array of overcurtailment in kW:\n", excessarray)
    print("\n")

    #parse to loads
    L0 = x[0:timeslotrange]
    L1 = x[timeslotrange:(timeslotrange*2)]
    L2 = x[(timeslotrange*2):(timeslotrange*3)]
    L3 = x[(timeslotrange*3):(timeslotrange*4)]
    L4 = x[(timeslotrange*4):(timeslotrange*5)]
    L5 = x[(timeslotrange*5):(timeslotrange*6)]
    L6 = x[(timeslotrange*6):(timeslotrange*7)]
    L7 = x[(timeslotrange*7):(timeslotrange*8)]
    L8 = x[(timeslotrange*8):(timeslotrange*9)]
    L9 = x[(timeslotrange*9):(timeslotrange*10)]
    L10 = x[(timeslotrange*10):(timeslotrange*11)]
    L11 = x[(timeslotrange*11):(timeslotrange*12)]
    L12 = x[(timeslotrange*12):(timeslotrange*13)]
    L13 = x[(timeslotrange*13):(timeslotrange*14)]
    L14 = x[(timeslotrange*14):(timeslotrange*15)]
    L15 = x[(timeslotrange*15):(timeslotrange*16)]
    L16 = x[(timeslotrange*16):(timeslotrange*17)]
    L17 = x[(timeslotrange*17):(timeslotrange*18)]
    L18 = x[(timeslotrange*18):(timeslotrange*19)]
    L19 = x[(timeslotrange*19):(timeslotrange*20)]
    
    #minimum on time
    minonL0 = 2
    minonL1 = 2
    minonL2 = 2
    minonL3 = 2
    minonL4 = 2
    minonL5 = 2
    minonL6 = 1
    minonL7 = 1
    minonL8 = 1
    minonL9 = 1
    minonL10 = 1
    minonL11 = 3
    minonL12 = 3
    minonL13 = 3
    minonL14 = 3
    minonL15 = 3
    minonL16 = 2
    minonL17 = 2
    minonL18 = 2
    minonL19 = 2
    
    #maximum off time
    maxoffL0 = 4
    maxoffL1 = 4
    maxoffL2 = 4
    maxoffL3 = 4
    maxoffL4 = 4
    maxoffL5 = 4
    maxoffL6 = 3
    maxoffL7 = 3
    maxoffL8 = 3
    maxoffL9 = 4
    maxoffL10 = 3
    maxoffL11 = 3
    maxoffL12 = 4
    maxoffL13 = 3
    maxoffL14 = 3
    maxoffL15 = 3
    maxoffL16 = 4
    maxoffL17 = 4
    maxoffL18 = 4
    maxoffL19 = 4
    
    #printing values
    ontimeL0 = np.count_nonzero(L0==1)
    offtimeL0 = 16 - ontimeL0
    print("L0 min on time:", minonL0)
    print("L0 on time:", ontimeL0)
    print("L0 max off time:", maxoffL0)
    print("L0 off time:", offtimeL0)
    print("\n")
    
    ontimeL1 = np.count_nonzero(L1==1)
    offtimeL1 = 16 - ontimeL1
    print("L1 min on time:", minonL1)
    print("L1 on time:", ontimeL1)
    print("L1 max off time:", maxoffL1)
    print("L1 off time:", offtimeL1)
    print("\n")
    
    ontimeL2 = np.count_nonzero(L2==1)
    offtimeL2 = 16 - ontimeL2
    print("L2 min on time:", minonL2)
    print("L2 on time:", ontimeL2)
    print("L2 max off time:", maxoffL2)
    print("L2 off time:", offtimeL2)
    print("\n")
    
    ontimeL3 = np.count_nonzero(L3==1)
    offtimeL3 = 16 - ontimeL3
    print("L3 min on time:", minonL3)
    print("L3 on time:", ontimeL3)
    print("L3 max off time:", maxoffL3)
    print("L3 off time:", offtimeL3)
    print("\n")
    
    ontimeL4 = np.count_nonzero(L4==1)
    offtimeL4 = 16 - ontimeL4
    print("L4 min on time:", minonL4)
    print("L4 on time:", ontimeL4)
    print("L4 max off time:", maxoffL4)
    print("L4 off time:", offtimeL4)
    print("\n")
    
    ontimeL5 = np.count_nonzero(L5==1)
    offtimeL5 = 16 - ontimeL5
    print("L5 min on time:", minonL5)
    print("L5 on time:", ontimeL5)
    print("L5 max off time:", maxoffL5)
    print("L5 off time:", offtimeL5)
    print("\n")
    
    ontimeL6 = np.count_nonzero(L6==1)
    offtimeL6 = 16 - ontimeL6
    print("L6 min on time:", minonL6)
    print("L6 on time:", ontimeL6)
    print("L6 max off time:", maxoffL6)
    print("L6 off time:", offtimeL6)
    print("\n")
    
    ontimeL7 = np.count_nonzero(L7==1)
    offtimeL7 = 16 - ontimeL7
    print("L7 min on time:", minonL7)
    print("L7 on time:", ontimeL7)
    print("L7 max off time:", maxoffL7)
    print("L7 off time:", offtimeL7)
    print("\n")
    
    ontimeL8 = np.count_nonzero(L8==1)
    offtimeL8 = 16 - ontimeL8
    print("L8 min on time:", minonL8)
    print("L8 on time:", ontimeL8)
    print("L8 max off time:", maxoffL8)
    print("L8 off time:", offtimeL8)
    print("\n")
    
    ontimeL9 = np.count_nonzero(L9==1)
    offtimeL9 = 16 - ontimeL9
    print("L9 min on time:", minonL9)
    print("L9 on time:", ontimeL9)
    print("L9 max off time:", maxoffL9)
    print("L9 off time:", offtimeL9)
    print("\n")
    
    ontimeL10 = np.count_nonzero(L10==1)
    offtimeL10 = 16 - ontimeL10
    print("L10 min on time:", minonL10)
    print("L10 on time:", ontimeL10)
    print("L10 max off time:", maxoffL10)
    print("L10 off time:", offtimeL10)
    print("\n")
    
    ontimeL11 = np.count_nonzero(L11==1)
    offtimeL11 = 16 - ontimeL11
    print("L11 min on time:", minonL11)
    print("L11 on time:", ontimeL11)
    print("L11 max off time:", maxoffL11)
    print("L11 off time:", offtimeL11)
    print("\n")
    
    ontimeL12 = np.count_nonzero(L12==1)
    offtimeL12 = 16 - ontimeL12
    print("L12 min on time:", minonL12)
    print("L12 on time:", ontimeL12)
    print("L12 max off time:", maxoffL12)
    print("L12 off time:", offtimeL12)
    print("\n")
    
    ontimeL13 = np.count_nonzero(L13==1)
    offtimeL13 = 16 - ontimeL13
    print("L13 min on time:", minonL13)
    print("L13 on time:", ontimeL13)
    print("L13 max off time:", maxoffL13)
    print("L13 off time:", offtimeL13)
    print("\n")
    
    ontimeL14 = np.count_nonzero(L14==1)
    offtimeL14 = 16 - ontimeL14
    print("L14 min on time:", minonL14)
    print("L14 on time:", ontimeL14)
    print("L14 max off time:", maxoffL14)
    print("L14 off time:", offtimeL14)
    print("\n")
    
    ontimeL15 = np.count_nonzero(L15==1)
    offtimeL15 = 16 - ontimeL15
    print("L15 min on time:", minonL15)
    print("L15 on time:", ontimeL15)
    print("L15 max off time:", maxoffL15)
    print("L15 off time:", offtimeL15)
    print("\n")
    
    ontimeL16 = np.count_nonzero(L16==1)
    offtimeL16 = 16 - ontimeL16
    print("L16 min on time:", minonL16)
    print("L16 on time:", ontimeL16)
    print("L16 max off time:", maxoffL16)
    print("L16 off time:", offtimeL16)
    print("\n")
    
    ontimeL17 = np.count_nonzero(L17==1)
    offtimeL17 = 16 - ontimeL17
    print("L17 min on time:", minonL17)
    print("L17 on time:", ontimeL17)
    print("L17 max off time:", maxoffL17)
    print("L17 off time:", offtimeL17)
    print("\n")
    
    ontimeL18 = np.count_nonzero(L18==1)
    offtimeL18 = 16 - ontimeL18
    print("L18 min on time:", minonL18)
    print("L18 on time:", ontimeL18)
    print("L18 max off time:", maxoffL18)
    print("L18 off time:", offtimeL18)
    print("\n")
    
    ontimeL19 = np.count_nonzero(L19==1)
    offtimeL19 = 16 - ontimeL19
    print("L19 min on time:", minonL19)
    print("L19 on time:", ontimeL19)
    print("L19 max off time:", maxoffL19)
    print("L19 off time:", offtimeL19)
    
    return

In [13]:
#print("Initial Value current cost:\n",my_swarm.current_cost)
#print("Initial Value current pos:\n",my_swarm.position)
#print("Initial Value pbest pos:\n",my_swarm.pbest_pos)
#print("Initial Value pbest cost:\n",my_swarm.pbest_cost)

iterations = 500 # Set 500 iterations
start = timeit.default_timer()
for i in range(iterations):
    
    # Part 0: Compute current cost
    j=0
    for j in range(my_swarm.n_particles):
        my_swarm.current_cost[j] = obj_func(my_swarm.position[j][:]) # Compute current cost (using obj func)
        temp_pbest_cost[j] = obj_func(my_swarm.pbest_pos[j][:])  # Compute personal best cost (using obj func)
    
    #my_swarm.pbest_cost = f(my_swarm.pbest_pos) # Compute current cost (using built in sphere func)
    #print("-------------------------------")
    #print("Pbest pos:\n",my_swarm.pbest_pos)
    #print("Pbest cost:\n",my_swarm.pbest_cost)
    #print("Current pos:\n",my_swarm.position)
    #print("Current cost:\n", my_swarm.current_cost)
    #print("Current velocity:\n", my_swarm.velocity)
   
   
    #Update personal best pos/cost 
    my_swarm.pbest_pos, my_swarm.pbest_cost = _compute_pbest(my_swarm,temp_pbest_cost) # Update and store
    
    # Part 2: Update global best
    # Note that gbest computation is dependent on your topology
    if np.min(my_swarm.pbest_cost) < my_swarm.best_cost:
        my_swarm.best_pos, my_swarm.best_cost = my_topology.compute_gbest(my_swarm)

    # Let's print our output
    if i%100==0:
        print('Iteration: {} | my_swarm.best_cost: {:.4f}'.format(i+1, my_swarm.best_cost))
        #print("Pbest cost:\n",my_swarm.pbest_cost)
        #print("Pbest pos:\n",my_swarm.pbest_pos)
        #print("Current velocity:\n", my_swarm.velocity)
        
    # Part 3: Update position and velocity matrices
    # Note that position and velocity updates are dependent on your topology
    
    #Fix the velocity and position computation for Binary application
    my_swarm.velocity = _compute_velocity(my_swarm,c1,c2,w)
    my_swarm.position = _compute_position(my_swarm) 
    

print('The best cost found by our swarm is: {:.4f}'.format(my_swarm.best_cost))
stop = timeit.default_timer()
print('Total Runtime: ', stop - start)
#print('The best position found by our swarm is: {}'.format(my_swarm.best_pos))

#divide array into 10 loads x 16 timeslots
timeslotrange = 16
L0 = my_swarm.best_pos[0:timeslotrange]
L1 = my_swarm.best_pos[timeslotrange:(timeslotrange*2)]
L2 = my_swarm.best_pos[(timeslotrange*2):(timeslotrange*3)]
L3 = my_swarm.best_pos[(timeslotrange*3):(timeslotrange*4)]
L4 = my_swarm.best_pos[(timeslotrange*4):(timeslotrange*5)]
L5 = my_swarm.best_pos[(timeslotrange*5):(timeslotrange*6)]
L6 = my_swarm.best_pos[(timeslotrange*6):(timeslotrange*7)]
L7 = my_swarm.best_pos[(timeslotrange*7):(timeslotrange*8)]
L8 = my_swarm.best_pos[(timeslotrange*8):(timeslotrange*9)]
L9 = my_swarm.best_pos[(timeslotrange*9):(timeslotrange*10)]
L10 = my_swarm.best_pos[(timeslotrange*10):(timeslotrange*11)]
L11 = my_swarm.best_pos[(timeslotrange*11):(timeslotrange*12)]
L12 = my_swarm.best_pos[(timeslotrange*12):(timeslotrange*13)]
L13 = my_swarm.best_pos[(timeslotrange*13):(timeslotrange*14)]
L14 = my_swarm.best_pos[(timeslotrange*14):(timeslotrange*15)]
L15 = my_swarm.best_pos[(timeslotrange*15):(timeslotrange*16)]
L16 = my_swarm.best_pos[(timeslotrange*16):(timeslotrange*17)]
L17 = my_swarm.best_pos[(timeslotrange*17):(timeslotrange*18)]
L18 = my_swarm.best_pos[(timeslotrange*18):(timeslotrange*19)]
L19 = my_swarm.best_pos[(timeslotrange*19):(timeslotrange*20)]

print('Schedule of Load 0:', L0)
print('Schedule of Load 1:', L1)
print('Schedule of Load 2:', L2)
print('Schedule of Load 3:', L3)
print('Schedule of Load 4:', L4)
print('Schedule of Load 5:', L5)
print('Schedule of Load 6:', L6)
print('Schedule of Load 7:', L7)
print('Schedule of Load 8:', L8)
print('Schedule of Load 9:', L9)
print('Schedule of Load 10:', L10)
print('Schedule of Load 11:', L11)
print('Schedule of Load 12:', L12)
print('Schedule of Load 13:', L13)
print('Schedule of Load 14:', L14)
print('Schedule of Load 15:', L15)
print('Schedule of Load 16:', L16)
print('Schedule of Load 17:', L17)
print('Schedule of Load 18:', L18)
print('Schedule of Load 19:', L19)
print("\n")

#checker for constraints
checkvalidity(my_swarm.best_pos)



Iteration: 1 | my_swarm.best_cost: 16440900.0000
Iteration: 101 | my_swarm.best_cost: 3402000.0000
Iteration: 201 | my_swarm.best_cost: 3402000.0000
Iteration: 301 | my_swarm.best_cost: 3402000.0000
Iteration: 401 | my_swarm.best_cost: 3402000.0000
The best cost found by our swarm is: 3402000.0000
Total Runtime:  76.3786281
Schedule of Load 0: [1 1 1 0 1 1 1 0 1 1 0 1 1 1 1 1]
Schedule of Load 1: [1 1 1 0 1 1 1 1 1 1 1 1 0 1 0 0]
Schedule of Load 2: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Schedule of Load 3: [1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1]
Schedule of Load 4: [1 1 1 1 1 1 0 1 1 0 0 1 1 1 0 1]
Schedule of Load 5: [1 1 1 1 0 1 1 1 1 0 1 0 1 1 1 1]
Schedule of Load 6: [1 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1]
Schedule of Load 7: [1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Schedule of Load 8: [1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1]
Schedule of Load 9: [1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1]
Schedule of Load 10: [0 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1]
Schedule of Load 11: [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1]
Schedule of Load 12: [1 